# Game Review Analysis Code
This notebook contains the Python code used to analyze sentiment and features across game reviews.

## 1. Feature Mention Frequency and Sentiment

In [ ]:
features = [
    'multiplayer_mentioned', 'bugs_mentioned', 'graphics_mentioned', 'story_mentioned',
    'controls_mentioned', 'ai_mentioned', 'updates_mentioned', 'price_mentioned'
]
feature_freq = df[features].sum().sort_values(ascending=False).reset_index()
feature_freq.columns = ['feature', 'mention_count']
feature_sentiment = {
    feature: df[df[feature]]['polarity'].mean() for feature in features
}
feature_sentiment_df = pd.DataFrame.from_dict(feature_sentiment, orient='index', columns=['avg_polarity']).reset_index()
feature_sentiment_df.columns = ['feature', 'avg_polarity']
feature_analysis_df = feature_freq.merge(feature_sentiment_df, on='feature')

## 2. Before vs After Feature Mentions and Sentiment

In [ ]:
feature_release_trends = []
for feature in features:
    before_count = df[(df[feature]) & (df['commented'] == 'before')].shape[0]
    after_count = df[(df[feature]) & (df['commented'] == 'after')].shape[0]
    before_polarity = df[(df[feature]) & (df['commented'] == 'before')]['polarity'].mean()
    after_polarity = df[(df[feature]) & (df['commented'] == 'after')]['polarity'].mean()
    feature_release_trends.append({
        'feature': feature,
        'before_mentions': before_count,
        'after_mentions': after_count,
        'before_avg_polarity': before_polarity,
        'after_avg_polarity': after_polarity
    })
release_analysis_df = pd.DataFrame(feature_release_trends)

## 3. Monthly Normalized Feature Trends

In [ ]:
df['month'] = df['commented_date'].dt.to_period("M").dt.to_timestamp()
monthly_feature_counts = df.groupby("month")[features].sum().reset_index()
monthly_feature_trends = monthly_feature_counts.melt(id_vars='month', var_name='feature', value_name='mention_count')
monthly_total_comments = df.groupby('month')["comment"].count().reset_index(name="total_comments")
monthly_feature_trends = monthly_feature_trends.merge(monthly_total_comments, on="month")
monthly_feature_trends['mentions_per_1000_comments'] = 1000 * monthly_feature_trends['mention_count'] / monthly_feature_trends['total_comments']

## 4. Game-Level Normalized Feature Mentions

In [ ]:
game_feature_counts = df.groupby("game")[features].sum()
game_comment_counts = df.groupby("game")["comment"].count()
game_feature_density = game_feature_counts.div(game_comment_counts, axis=0)
game_feature_density["genre"] = df.groupby("game")["genre"].first()